In [ ]:
import numpy as np
from random import choices, randrange
import random
import matplotlib.pyplot as plt
from numba import njit
from sklearn.datasets import make_blobs
from utility import *
from tqdm import tqdm

Implementazione del simulated annealing. La soluzione candidato è costruita effettuando lo spostmento di un elemento da un cluster ad un altro.

In [ ]:
@njit
def change_sol(sol, k):
    new_sol = sol.copy()
    n = randrange(len(new_sol))
    old_value = new_sol[n]

    while old_value == new_sol[n]:
        new_sol[n] = randrange(K)

    return new_sol

@njit
def simulated_annealing(base_sol, points, K, iters, alpha, Ti = 10, Tf = 10**-8, verbose = True):
    best_sol = base_sol
    base_val = squared_inner_distance(best_sol, points, K)
    best_value = base_val
    T = Ti
    curr_sol = base_sol
    curr_val = best_value
    
    iter = 1
    old_sol = base_sol
    finito = False
    no_update = 0

    while(finito == False):
        if(verbose and iter%10 == 1):
            print("Iteration number:", iter, "Best value percentuale: ", curr_val/base_val*100, "% T:", T, no_update)
        
        iter = 1 + iter
        old_sol = curr_sol
        
        for i in range(iters):
            candidate = change_sol(curr_sol, K)
            val_candidate = squared_inner_distance(candidate, points, K)

            if(val_candidate < best_value):
                best_value = val_candidate
                best_sol = candidate
            
            if(val_candidate < curr_val):
                curr_val = val_candidate
                curr_sol = candidate
                no_update = 0
            else:
                r = random.uniform(0, 1)
                delta = abs(curr_val - val_candidate)
                tresh = np.exp(-delta/T)
                if(r < tresh):
                    curr_val = val_candidate
                    curr_sol = candidate
                    no_update = 0
                else:
                    no_update += 1

        if(no_update >= 10000 or T < Tf):
            finito = True
            break
        
        T = alpha*T
    return best_sol

In [ ]:
N = 1000
K = 10

points, centroids = make_blobs(n_samples=N, centers=K, n_features=2, random_state=np.random.randint(100))

sol = create_initial_sol(points, K)
sol = simulated_annealing(sol, points, K, 100, 0.99, 100, 0.001)

print("{:.5E}".format(squared_inner_distance(sol, points, K)))

printR2sol(points, sol, K)